<a href="https://colab.research.google.com/github/dhirajsuvarna/pointnet.pytorch/blob/master/notebooks/PointNet_Training_DMU_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement PointNet on DMU-Dataset - Segmentation



### Setup the google drive with dataset and code

In [4]:
from google.colab import drive
drive.mount('/content/mydrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mydrive


In [0]:
cd /content/mydrive/My\ Drive/Projects/pointnet

/content/mydrive/My Drive/Projects/pointnet


In [0]:
cd pointnet.pytorch/

/content/mydrive/My Drive/Projects/pointnet/pointnet.pytorch


In [5]:
#for the first time
#!git clone https://github.com/dhirajsuvarna/pointnet.pytorch.git
! git pull

remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 23 (delta 10), reused 18 (delta 8), pack-reused 0
Unpacking objects: 100% (23/23), done.
From https://github.com/dhirajsuvarna/pointnet.pytorch
   3fff6f3..278b69d  master     -> origin/master
Updating 3fff6f3..278b69d
Fast-forward
  notebooks/PointNet_Training_DMU.ipynb           | 7574 ++++++++++++++++++++++
 dataset_utils/pcd_utils/file_format_converter.py |   38 +
 infer.py                                         |   84 +
 train_segmentation.py                            |  142 +
 4 files changed, 7838 insertions(+)
 create mode 100644  notebooks/PointNet_Training_DMU.ipynb
 create mode 100644 dataset_utils/pcd_utils/file_format_converter.py
 create mode 100644 infer.py
 create mode 100644 train_segmentation.py


In [0]:
!git config --global user.email "dhiraj.suvarna@gmail.com"

In [0]:
! git stash

Saved working directory and index state WIP on master: fd5a674 Merge branch 'master' of https://github.com/dhirajsuvarna/pointnet.pytorch


In [0]:
cd /content/mydrive/My\ Drive/Projects/pointnet/dataset

/content/mydrive/My Drive/Projects/pointnet/dataset


In [0]:
!unzip dataset_PCD_5000.zip

### try to get max gpu

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=8dd00f879db4783d97f015fce06cedbac1416865705d6f01127a98baa73f7f4d
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 158.5 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


# Install the Python Pacakages

In [5]:
cd /content/mydrive/My\ Drive/Projects/pointnet/pointnet.pytorch


/content/mydrive/My Drive/Projects/pointnet/pointnet.pytorch


In [6]:
!pip install plyfile

In [7]:
!python -m pip install git+https://github.com/DanielPollithy/pypcd.git

  Cloning https://github.com/DanielPollithy/pypcd.git to /tmp/pip-req-build-hc75fjfj
  Running command git clone -q https://github.com/DanielPollithy/pypcd.git /tmp/pip-req-build-hc75fjfj
  Created wheel for pypcd: filename=pypcd-0.1.3-py2.py3-none-any.whl size=23397 sha256=1e06b803661b0c7893e5fe549dc188e1f331b2e9557f0626f8f3252045c1ec66
  Stored in directory: /tmp/pip-ephem-wheel-cache-zkj7w0ck/wheels/1a/a3/7a/62170df4ad89eaaf95e7c9d8ee64808580cde1b9f8c22d8562
  Created wheel for python-lzf: filename=python_lzf-0.2.4-cp36-cp36m-linux_x86_64.whl size=22076 sha256=9979210fbc38c7997d9bf4de25b27da82eddc97090db3578e972ccb4ca209c98
  Stored in directory: /root/.cache/pip/wheels/67/92/2e/de0656a80f99649ca610c10d96aef9aed3ef19697606bcb94f
Successfully built pypcd python-lzf


### Create Train Test Split

In [0]:
!python create_train_test_shuffle.py

debug: Actuator => totalFiles: 38, train: 30 + test: 8 = 38 
debug: AirFilter => totalFiles: 32, train: 25 + test: 7 = 32 
debug: Battery => totalFiles: 46, train: 36 + test: 10 = 46 
debug: Bearing => totalFiles: 128, train: 102 + test: 26 = 128 
debug: Brake => totalFiles: 39, train: 31 + test: 8 = 39 
debug: Camshaft => totalFiles: 46, train: 36 + test: 10 = 46 
debug: Clutch => totalFiles: 61, train: 48 + test: 13 = 61 
debug: Cooler => totalFiles: 30, train: 24 + test: 6 = 30 
debug: Coupling => totalFiles: 43, train: 34 + test: 9 = 43 
debug: Crankshaft => totalFiles: 81, train: 64 + test: 17 = 81 
debug: CuttingTool => totalFiles: 41, train: 32 + test: 9 = 41 
debug: ElectricMotor => totalFiles: 142, train: 113 + test: 29 = 142 
debug: ElectronicBoard => totalFiles: 37, train: 29 + test: 8 = 37 
debug: ExhaustPipe => totalFiles: 20, train: 16 + test: 4 = 20 
debug: Gear => totalFiles: 18, train: 14 + test: 4 = 18 
debug: GearWheel => totalFiles: 91, train: 72 + test: 19 = 91 
de

In [0]:
!cat dataset_PCD_5000/train_test_split.json | head 

{
    "train": {
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_15.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_27.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_21.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_12.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_35.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_13.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_32.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_17.pcd": "Actuator",


In [0]:
cd ../pointnet.pytorch/

/content/mydrive/My Drive/Projects/pointnet/pointnet.pytorch


# Start Training 

In [0]:
!rm -r runs

### Running on Pre-exiting Dataset

In [2]:
pwd


'/content'

In [9]:
!python train_segmentation.py --nepoch 10 \
                                --batchSize 32 \
                                --dataset /content/mydrive/My\ Drive/Projects/pointnet/dataset/shapenetcore_partanno_segmentation_benchmark_v0

Namespace(batchSize=32, class_choice='Chair', dataset='/content/mydrive/My Drive/Projects/pointnet/dataset/shapenetcore_partanno_segmentation_benchmark_v0', feature_transform=False, model='', nepoch=10, outf='saved_models', workers=4)
Random Seed:  4356
{'Chair': 0}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4
{'Chair': 0}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4
2658 704
classes 4
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this